<center><img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px; margin-left: 10px; max-height: 250px;" /></center>

# 2.2 Data Discovery & Properties

## GDAL, STAC and Data Properties

In this exercise we interact with a SpatioTemporal Asset Catalog (STAC) and explore the metadata using GDAL.

Start importing the necessary libraries

In [ ]:
from osgeo import gdal
from pystac_client import Client

import rasterio
import numpy as np
import matplotlib.pyplot as plt

### Exploring STAC Collections

Connect to a STAC API and explore available data collections.  

In [ ]:
from pystac_client import Client

# Connect to a STAC API
catalog = Client.open("https://earth-search.aws.element84.com/v1")

# List available collections
collections = catalog.get_collections()
print("Available STAC Collections:")
for collection in collections:
    print(collection.id)

### Querying Sentinel-2 data from a STAC API

Connect to the same public STAC API and search for Sentinel-2 Level-2A products over a specific area (Sardinia) and time range (September 2023).  
We will filter the results to include only those items with less than or equal to 20% cloud cover.  
The code retrieves all matching items and prints the number of results, as well as the unique IDs of the returned items.  
This demonstrates how to query a geospatial data catalog programmatically using the `pystac-client` library- [which documentation can be found here](https://pystac-client.readthedocs.io/).


In [ ]:
# Connect to a public STAC API (e.g., Sentinel-2)
catalog = Client.open("https://earth-search.aws.element84.com/v1")

# Search for Sentinel-2 items over a specific area and date range
items = catalog.search(
    collections=["sentinel-2-l2a"],
    datetime="2023-09-01/2023-09-30",
    bbox=[8.0, 40.0, 9.0, 41.0],  # Example bounding box for Sardinia
    query={"eo:cloud_cover": {"lte": 20}}
).item_collection()

# Print the number of items found
print(f"Number of items found: {len(items)}\n")

# Display the IDs of the found items
print("List of Sentinel-2 ids:")
for item in items:
    print(item.id)

### Metadata exploration

Display some metadata of the first STAC Item returned by our previous query:

In [ ]:
# Display metadata for the found items
print(f"ID: {items[0].id}")
print(f"Date: {items[0].datetime}")
print(f"Cloud Cover: {items[0].properties['eo:cloud_cover']}%")
print(f"Geometry: {items[0].geometry}")
print(f"Projection as EPSG Code: {items[0].properties['proj:epsg']}")

### Accessing and inspecting a Sentinel-2 red band raster

Access the first Sentinel-2 item from a previously retrieved STAC search result and extract the URL for the red band.  
Using GDAL, we will open the raster file from this URL and inspect its metadata, including the raster size, coordinate reference system (CRS), and geotransform parameters.  
This demonstrates how to programmatically access specific assets within a STAC item and retrieve relevant geospatial metadata.

Inspect the structure of items, available assets and properties

In [ ]:
items[0]

Print the metadata using GDAL Python

In [ ]:
first_item = items[0]

asset_red_href = first_item.assets['red'].href  # Assuming B04 (red band) is available

# Open and inspect the raster file
dataset = gdal.Open(asset_red_href)
print(f"Raster Size: {dataset.RasterXSize} x {dataset.RasterYSize}")
print(f"Projection: {dataset.GetProjection()}")
geotransform = dataset.GetGeoTransform()
print(f"GeoTransform: {dataset.GetGeoTransform()}")

# Get spatial resolution
pixel_width = geotransform[1]
pixel_height = geotransform[5]
print(f"Pixel Size: {pixel_width} x {pixel_height}")

# Get the number of bands
bands = dataset.RasterCount
print(f"Number of Bands: {bands}")

Print the metadata using GDAL from command line

In [ ]:
import os
os.system(f"gdalinfo {first_item.assets['red'].href}")

### Simple band visualization

Read the red band data as a NumPy array:

In [ ]:
red_band = dataset.GetRasterBand(1).ReadAsArray()

Finally, visualize the content

In [ ]:
plt.imshow(red_band/1800,vmin=0,vmax=1)